# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
Vdf = pd.read_csv("../output_data/cities.csv")
Vdf

,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
0,Rikitea,PF,2020-05-06 22:48:01,-23.12,-134.97,78,66,74.84,12.19
1,Cape Town,ZA,2020-05-06 22:46:17,-33.93,18.42,29,93,59.00,3.36
2,Hermanus,ZA,2020-05-06 22:48:02,-34.42,19.23,37,78,61.79,9.19
3,Bluff,NZ,2020-05-06 22:48:02,-46.60,168.33,47,78,54.48,28.45
4,Saskylakh,RU,2020-05-06 22:48:02,71.92,114.08,74,93,5.94,7.56
...,...,...,...,...,...,...,...,...,...
565,Koubia,GN,2020-05-06 22:49:30,11.58,-11.90,85,61,76.08,3.15
566,Tupã,BR,2020-05-06 22:49:30,-21.93,-50.51,100,97,61.83,8.10
567,Ganzhou,CN,2020-05-06 22:49:30,28.82,115.43,91,81,78.21,1.36
568,Hikari,JP,2020-05-06 22:49:30,33.95,131.95,75,45,66.20,14.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps.
gmaps.configure(g_key)


In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = Vdf[["Latitude", "Longitude"]].astype(float)

# Use humidity as weight
humidity = Vdf["Humidity"].astype(float) 


# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#   * A max temperature lower than 75 degrees but higher than 50
loc_temp= (Vdf['Max Temperature'] > 50)&(Vdf['Max Temperature'] < 75)

#   * Wind speed less than 20 mph.

loc_wind_speed =Vdf['Wind Speed']<20
#   * Zero cloudiness.
loc_cloudiness = Vdf.Cloudiness <80
#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
filtered_weather_df = Vdf[loc_temp & loc_wind_speed&loc_cloudiness]
filtered_weather_df

,City,Country,Date,Latitude,Longitude,Cloudiness,Humidity,Max Temperature,Wind Speed
0,Rikitea,PF,2020-05-06 22:48:01,-23.12,-134.97,78,66,74.84,12.19
1,Cape Town,ZA,2020-05-06 22:46:17,-33.93,18.42,29,93,59.00,3.36
2,Hermanus,ZA,2020-05-06 22:48:02,-34.42,19.23,37,78,61.79,9.19
10,Kiryat Gat,IL,2020-05-06 22:48:03,31.61,34.76,0,79,59.00,3.00
12,Port Alfred,ZA,2020-05-06 22:48:03,-33.59,26.89,0,82,61.75,4.29
...,...,...,...,...,...,...,...,...,...
549,Caravelas,BR,2020-05-06 22:49:27,-17.71,-39.25,71,82,74.98,6.91
555,Dondo,MZ,2020-05-06 22:49:29,-19.61,34.74,76,86,66.31,5.57
560,Gari,RU,2020-05-06 22:49:29,59.43,62.35,2,58,62.31,5.75
564,Buin,CL,2020-05-06 22:49:30,-33.73,-70.75,15,28,55.40,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame([Vdf.City, Vdf.Country, Vdf.Latitude, Vdf.Longitude]).transpose()
hotel_df['Hotel Name']=''
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,Rikitea,PF,-23.12,-134.97,
1,Cape Town,ZA,-33.93,18.42,
2,Hermanus,ZA,-34.42,19.23,
3,Bluff,NZ,-46.6,168.33,
4,Saskylakh,RU,71.92,114.08,
...,...,...,...,...,...
565,Koubia,GN,11.58,-11.9,
566,Tupã,BR,-21.93,-50.51,
567,Ganzhou,CN,28.82,115.43,
568,Hikari,JP,33.95,131.95,


In [7]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
#     "location": f"{hotel_df.Latitude},{hotel_df.Longitude}",
    "radius":5000,
    "keyword":"Hotel",
    "type": "Hotel",
    "key": g_key
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # add keyword to params dict

    params['location'] = f"{row['Latitude']},{row['Longitude']}"

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(url, params=params).json()

    # extract results
    results = response['results']

    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("No Hotel found/result... skipping.")

Retrieving Results for Index 0: Rikitea.
Retrieving Results for Index 1: Cape Town.
Retrieving Results for Index 2: Hermanus.
Retrieving Results for Index 3: Bluff.
Retrieving Results for Index 4: Saskylakh.
No Hotel found/result... skipping.
Retrieving Results for Index 5: Nikolskoye.
Retrieving Results for Index 6: Ushuaia.
Retrieving Results for Index 7: North Myrtle Beach.
Retrieving Results for Index 8: Kindu.
Retrieving Results for Index 9: Sochaczew.
Retrieving Results for Index 10: Kiryat Gat.
Retrieving Results for Index 11: Faanui.
Retrieving Results for Index 12: Port Alfred.
Retrieving Results for Index 13: Sibolga.
Retrieving Results for Index 14: Atuona.
Retrieving Results for Index 15: Mahébourg.
Retrieving Results for Index 16: Tuktoyaktuk.
Retrieving Results for Index 17: Chokurdakh.
No Hotel found/result... skipping.
Retrieving Results for Index 18: Wasilla.
Retrieving Results for Index 19: Bud.
No Hotel found/result... skipping.
Retrieving Results for Index 20: North

Retrieving Results for Index 165: Nkhotakota.
Retrieving Results for Index 166: Teya.
No Hotel found/result... skipping.
Retrieving Results for Index 167: Cardston.
Retrieving Results for Index 168: Ribeira Grande.
Retrieving Results for Index 169: Caledon.
Retrieving Results for Index 170: Chapais.
Retrieving Results for Index 171: Madang.
Retrieving Results for Index 172: Carnarvon.
Retrieving Results for Index 173: Ponta Delgada.
Retrieving Results for Index 174: Souillac.
Retrieving Results for Index 175: Panzhihua.
Retrieving Results for Index 176: Bykovo.
Retrieving Results for Index 177: Hambantota.
Retrieving Results for Index 178: Ranau.
Retrieving Results for Index 179: Phuket.
Retrieving Results for Index 180: Nishihara.
Retrieving Results for Index 181: Mersing.
Retrieving Results for Index 182: Karratha.
Retrieving Results for Index 183: Zhigalovo.
Retrieving Results for Index 184: Karema.
No Hotel found/result... skipping.
Retrieving Results for Index 185: Port Hardy.
Ret

Retrieving Results for Index 331: Kirakira.
No Hotel found/result... skipping.
Retrieving Results for Index 332: La Ronge.
Retrieving Results for Index 333: Ploemeur.
Retrieving Results for Index 334: Morón.
Retrieving Results for Index 335: Plettenberg Bay.
Retrieving Results for Index 336: Kieta.
No Hotel found/result... skipping.
Retrieving Results for Index 337: Claveria.
Retrieving Results for Index 338: Maracaçumé.
Retrieving Results for Index 339: Maināguri.
No Hotel found/result... skipping.
Retrieving Results for Index 340: Keflavik.
Retrieving Results for Index 341: Acala.
No Hotel found/result... skipping.
Retrieving Results for Index 342: Constitución.
Retrieving Results for Index 343: Tautira.
No Hotel found/result... skipping.
Retrieving Results for Index 344: Saint-Pierre.
Retrieving Results for Index 345: Khuzdar.
No Hotel found/result... skipping.
Retrieving Results for Index 346: Tigil'.
Retrieving Results for Index 347: Iqaluit.
Retrieving Results for Index 348: Sald

Retrieving Results for Index 491: Huilong.
Retrieving Results for Index 492: Bundaberg.
Retrieving Results for Index 493: Ubari.
No Hotel found/result... skipping.
Retrieving Results for Index 494: Nemuro.
Retrieving Results for Index 495: Alamosa.
Retrieving Results for Index 496: Dzhebariki-Khaya.
No Hotel found/result... skipping.
Retrieving Results for Index 497: Troitskoye.
No Hotel found/result... skipping.
Retrieving Results for Index 498: Beyneu.
Retrieving Results for Index 499: Tessalit.
No Hotel found/result... skipping.
Retrieving Results for Index 500: Le Vauclin.
Retrieving Results for Index 501: Tombouctou.
No Hotel found/result... skipping.
Retrieving Results for Index 502: Mujiayingzi.
No Hotel found/result... skipping.
Retrieving Results for Index 503: Hamilton.
Retrieving Results for Index 504: Panacan.
Retrieving Results for Index 505: Tucupita.
Retrieving Results for Index 506: Menongue.
Retrieving Results for Index 507: Tagajo.
Retrieving Results for Index 508: Qo

In [8]:
hotel_df


,City,Country,Latitude,Longitude,Hotel Name
0,Rikitea,PF,-23.12,-134.97,Pension Maro'i
1,Cape Town,ZA,-33.93,18.42,Atlanticview Cape Town Boutique Hotel
2,Hermanus,ZA,-34.42,19.23,The Marine Hermanus
3,Bluff,NZ,-46.6,168.33,Foveaux Hotel
4,Saskylakh,RU,71.92,114.08,
...,...,...,...,...,...
565,Koubia,GN,11.58,-11.9,
566,Tupã,BR,-21.93,-50.51,Hotel Avenida
567,Ganzhou,CN,28.82,115.43,Chunlei Hotel
568,Hikari,JP,33.95,131.95,Kanponoyado Hikari


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
hotels_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(hotels_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))